# Artık Ağlar (Residual Network) (ResNet)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models # Önceden eğitilmiş modelleri içeri katarma
from tqdm import tqdm #İlerleme çubuğu

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### 1)Veri Yükleme

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(), # Görselleri tensore çevirme
    transforms.Normalize((0.5 , 0.5 , 0.5) , (0.5 , 0.5 , 0.5))
])

# CIFAR10 veri kümesi indir ve yükle
trainset = torchvision.datasets.CIFAR10( root = "./CIFAR10_data" ,train = True, download = True , transform = transform )
testset = torchvision.datasets.CIFAR10( root = "./CIFAR10_data" , train = False, download = True , transform = transform )

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset , batch_size = 64 , shuffle = True)
test_loader = torch.utils.data.DataLoader(testset , batch_size = 64 , shuffle = False)

100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [01:10<00:00, 2.41MB/s]


### 2)Residual Blokların Hazırlanması

In [13]:
class ResidualBlock(nn.Module):

    def __init__(self,in_channels , out_channels , stride = 1 , downsample = None):
        super(ResidualBlock,self).__init__()
        """
            conv2d -> BatchNorm -> ReLu -> conv2d -> BatchNorm -> Downsampling
        """

        # 3x3 conv2d
        self.conv1 = nn.Conv2d(in_channels=in_channels , out_channels = out_channels , kernel_size = 3 ,stride = stride , padding = 1 , bias = False)

        # Batch Normalization
        self.bn1 = nn.BatchNorm2d(out_channels)

        # ReLu aktivasyon fonk.
        self.relu = nn.ReLU()

        # 3x3 conv2d
        self.conv2 = nn.Conv2d(out_channels , out_channels , kernel_size = 3 , stride = 1 , padding = 1 , bias = False)

        # Batch Normalization Layer
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = downsample
    
    def forward(self,x):
        identity = x # Kendi kendine bağlanacak giriş verisi

        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x) # İlk conv katmanı
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = out + identity # Skip connection
        out = self.relu(out)
        
        return out

### 3)ResNet Oluşturma (Custom)

In [14]:
class CustomResNet(nn.Module):
    
    def __init__(self,num_classes = 10):
        """
            conv2d -> BatchNorm -> ReLu -> MaxPool -> 4 x Layer -> Avgpool -> fc 
        """
        super(CustomResNet,self).__init__()

        # İlk Conv
        self.conv1 = nn.Conv2d(3,64,kernel_size = 7 , stride = 2 , padding = 3 , bias = False)

        # Batch Normalization
        self.bn1 = nn.BatchNorm2d(64)

        # ReLU
        self.relu = nn.ReLU()

        # Max Pooling
        self.maxpool = nn.MaxPool2d(kernel_size = 3 , stride = 2 , padding = 1)

        # 4 x make layer
        self.layer1 = self._make_layer(64,64,2)
        self.layer2 = self._make_layer(64,128,2,2)
        self.layer3 = self._make_layer(128,256,2,2)
        self.layer4 = self._make_layer(256,512,2,2)

        # AVG - Pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        # Tam Bağlı Layer
        self.fc = nn.Linear(512,num_classes)

    def _make_layer(self , in_channels , out_channels , blocks , stride = 1): # Residual katmanları oluşturan fonk.
        downsample = None

        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels , out_channels , kernel_size = 1 , stride = stride , bias = False),
                nn.BatchNorm2d(out_channels)
            )

        # İlk Residual bloklar
        layers = [ResidualBlock(in_channels , out_channels , stride , downsample)]
        
        # Sonraki residual bloklar
        for _ in range(1,blocks):
            layers.append(ResidualBlock(out_channels , out_channels))
        return nn.Sequential(*layers)

    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # ResNet bloklarının sırayla uygulanması
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.fc(x)

        return x

In [8]:
# model = CustomResNet()

### 4)Resnet ile Transfer Learning ve Custom Resnet ile Training

In [17]:
use_custom_model = True

if use_custom_model:
    model = CustomResNet().to(device)

else:
    model = models.resnet18(pretrained = True) # Hazır resnet 18 modeli ile fine tuning
    num_ftrs = model.fc.in_features
    # Sınıflandırma blok 
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs , 256),
        nn.ReLU(),
        nn.Linear(256,10) # Output Layer
    )
    model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr = 0.001)

# Modelin Eğitilmesi
num_epochs = 1
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0

    for images , labels in tqdm(train_loader):
        images , labels = images.to(device) , labels.to(device)
        optimizer.zero_grad() # Gradyanları sıfırlama
        outputs = model(images) # Modeli ileri besleme
        loss = criterion(outputs , labels) # Loss hesaplama
        loss.backward() # Geri yayılım
        optimizer.step() # Ağırlık güncelleme (weight update)
        running_loss += loss.item()
    print(f"Epoch : {epoch+1}/{num_epochs} - Loss : {running_loss / len(train_loader)}")

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [05:39<00:00, 339.23s/it]

Epoch : 1/1 - Loss : 1.380485955025534


### 5)Modelin Test Edilmesi

In [18]:
model.eval()
correct = 0 
total = 0 
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test accuracy: {100 * correct / total}%")

Test accuracy: 52.28%
